In [1]:
import torch
import torch.nn as nn

In [2]:
class BasicBlock(nn.Module):
    expansion=1
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, stride=stride, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels*BasicBlock.expansion)
        )
        self.shortcut=nn.Sequential()
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion))
    def forward(self,x):
        return nn.ReLU(inplace=True)(self.residual_function(x)+self.shortcut(x))

In [3]:
def dimension_check():
    net=BasicBlock(3,3,2)
    print(net.residual_function)
    print(net.shortcut)
    x= torch.randn(1,3,50,50)
    y=net(x)
    print(y.size())

In [4]:
dimension_check()

Sequential(
  (0): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv2d(3, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (4): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
Sequential(
  (0): Conv2d(3, 3, kernel_size=(1, 1), stride=(2, 2), bias=False)
  (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
torch.Size([1, 3, 25, 25])


In [5]:
class BottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.residual_function = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, stride=stride, kernel_size=3, padding=1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size=1, bias=False),
        nn.BatchNorm2d(out_channels * BottleNeck.expansion),
        )
        
        self.shortcut = nn.Sequential()
        
        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BottleNeck.expansion, stride=stride, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_channels * BottleNeck.expansion))
    def forward(self, x):
        return nn.ReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))
            

In [6]:
def dimension_check():
    net=BottleNeck(3,3,1)
    print(net.residual_function)
    print(net.shortcut)
    x= torch.randn(1,3,50,50)
    y=net(x)
    print(y.size())

In [7]:
dimension_check()

Sequential(
  (0): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (4): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): Conv2d(3, 12, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (7): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
Sequential(
  (0): Conv2d(3, 12, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
torch.Size([1, 12, 50, 50])


In [8]:
class ResNet(nn.Module):
    
    def __init__(self, block, num_block, num_classes=2):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True))
        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        output = self.conv3_x(output)
        output = self.conv4_x(output)
        output = self.conv5_x(output)
        output = self.avg_pool(output)
        output = output.view(output.size(0), -1)
        output = self.fc(output) # => 이미지 크기가 
                                 #바뀌면 이 줄 코드를 # 하고 dimension 확인

        return output

In [9]:
def resnet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])
def resnet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])
def resnet50():
    return ResNet(BottleNeck, [3, 4, 6, 3])
def resnet101():
    return ResNet(BottleNeck, [3, 4, 23, 3])
def resnet152():
    return ResNet(BottleNeck, [3, 8, 36, 3])

In [10]:
def dimension_check():
    net=resnet50()
    x= torch.randn(1,3,50,50) # 28 이미지
    y=net(x)
    print(y.size())

In [11]:
dimension_check()

torch.Size([1, 2])


In [12]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy
import seaborn as sns 
import matplotlib.pyplot as plt

In [13]:
Porn=np.load("Porn_data.npy")
Non_Porn=np.load("Non_Porn_data.npy")

In [14]:
input_data=[]
for i in range(2):
    if i == 0:
        for j in range(len(Non_Porn)):
            input_data.append(Non_Porn[j])
    else :
        for j in range(len(Porn)):
            input_data.append(Porn[j])

In [15]:
input_data=np.array(input_data)

In [16]:
label=[]
for i in range(len(Non_Porn)+len(Porn)):
    if i>len(Non_Porn):
        label.append(0)
    else:
        label.append(1)

In [17]:
label=np.array(label)

In [18]:
input_data= (input_data*(1/256)-0.5)/0.5

In [19]:
input_data=torch.FloatTensor(input_data)
label=torch.LongTensor(label)

In [20]:
input_data=input_data.transpose(1,3)

In [21]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
dataset = TensorDataset(input_data,label)

In [22]:
trainset,valset=torch.utils.data.random_split(dataset,[116681,20000])
trainset,testset=torch.utils.data.random_split(trainset,[96681,20000])

partition={'train':trainset,'val':valset,'test':testset}

In [23]:
def train(net,partition,optimizer,criterion):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size=256,
                                             shuffle=True)
    net.train()
    correct = 0
    total =0
    train_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, 0)):
        time.sleep(0.0000001)
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        #break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #if i % 200 == 0:
        #    print(correct,train_loss)
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [24]:
def test(net):
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=256, 
                                             shuffle=False, num_workers=2)
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [25]:
def validate(net, partition, criterion):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=256, 
                                            shuffle=False, num_workers=2)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in tqdm(valloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [26]:
from tqdm import tqdm, tqdm_notebook
from tqdm import trange
import time

In [27]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
net = resnet34()
net.cuda()
criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0015, weight_decay=0.00001)
train_losses=[]
val_losses=[]
train_accs=[]
val_accs=[]
for epoch in range(100):
    ts=time.time()
    net, train_loss,train_acc=train(net, partition, optimizer, criterion)
    val_loss, val_acc = validate(net, partition, criterion)
    te=time.time()
    print('time : {:2.2f}, train_loss : {:2.2f}, train_acc : {:2.2f},val_acc : {:2.2f},val_loss : {:2.2f}'.format(te-ts,train_loss,train_acc,val_acc,val_loss))
    train_loss=round(train_loss,2)
    val_loss=round(val_loss,2)
    train_acc=round(train_acc,2)
    val_acc=round(val_acc,2)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

  0%|          | 0/378 [00:00<?, ?it/s]

time : 151.19, train_loss : 0.31, train_acc : 87.90,val_acc : 88.17,val_loss : 0.30


  0%|          | 0/378 [00:00<?, ?it/s]

time : 153.20, train_loss : 0.21, train_acc : 92.13,val_acc : 91.81,val_loss : 0.21


  0%|          | 0/378 [00:00<?, ?it/s]

...

time : 150.90, train_loss : 0.01, train_acc : 99.74,val_acc : 98.45,val_loss : 0.06


  0%|          | 0/378 [00:00<?, ?it/s]

time : 150.93, train_loss : 0.01, train_acc : 99.67,val_acc : 98.81,val_loss : 0.05


100%|██████████| 79/79 [00:08<00:00,  9.65it/s]

time : 150.91, train_loss : 0.01, train_acc : 99.81,val_acc : 98.84,val_loss : 0.05


In [29]:
test_acc=test(net)

100%|██████████| 79/79 [00:07<00:00, 10.09it/s]


In [30]:
test_acc

98.975

In [32]:
np.save("ResNet_trainloss",train_losses)
np.save("ResNet_valloss",val_losses)
np.save("ResNet_trainaccs",train_accs)
np.save("ResNet_valloss",val_accs)

In [33]:
vggtri=np.load("ResNet_trainloss.npy")

In [ ]:
vggtri